# Assignment 1 Multiclass SVM 을 직접 구현
https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

1.
위에서 언급 되었던 **Multiclass SVM** 을 직접 구현하시는 것입니다.  
기본적으로 사이킷 런에 있는 SVM 은 멀티클래스 SVM 을 지원합니다.  
그러나 과제에서는 그것을 쓰면 안됩니다!  
아이리스 데이터는 총 세 개의 클래스가 있으므로 **이 클래스를 one hot 인코딩** 한 뒤,   
각각 **binary SVM 을 트레이닝**하고 이 **결과를 조합**하여 multiclass SVM 을 구현하는 것입니다.  

2.
위에서 말했듯 기본적으로 **one vs one, one vs rest** 방법이 있으며 어떤 것을 구현하든 자유입니다.   
만약 투표결과 동점이 나온경우 예를 들어 각각의 SVM 의 결과가 A vs B C 의 경우 A 로 판별 , B vs A C 의 결과 B 로 판별 , C vs A B 의 경우 C 로 판별한 경우  
투표를 통해 class 를 결정할 수 없는 경우 **decision_function** 을 활용하시거나,   
**가장 개수가 많은 클래스를 사용**하시거나 **랜덤**으로 하나를 뽑거나 하는 방법 등을 이용해 동점자인 경우를 판별해주시면 됩니다.  
공식 문서를 보면 사이킷런이 어떤 방법으로 구현했는지가 글로 나와 있으므로 참조하셔도 무관합니다.  

3.
과제코드에는 제가 iris 데이터를 로드하고 iris 데이터를 one hot 인코딩 한 부분까지 구현해 놓았습니다  
또한 decision function 을 호출해서 사용하는 예시도 하나 넣어 놓았으니 참고하시면 됩니다  
개인적으로 one vs rest 가 더 구현하기 쉬울것으로 생각되며, 모르는 부분은 언제든 질문해주세요!   
생각보다 코드가 길지 않고 어렵지 않습니다.  

---

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
iris =  sns.load_dataset('iris') #data load
X = iris.iloc[:,:4]
y = iris.iloc[:,-1]

In [3]:
X.head()

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [4]:
y.head()

0    setosa
1    setosa
2    setosa
3    setosa
4    setosa
Name: species, dtype: object

In [5]:
# target의 분포 동일하다.
y.value_counts()

setosa        50
versicolor    50
virginica     50
Name: species, dtype: int64

In [6]:
def one_rest_svm(kernel, gamma, C, X, y):
    # One Versus Rest (각각의 model training)
    svm_setosa     = SVC(kernel =kernel, gamma = gamma, C = C)
    svm_versicolor = SVC(kernel =kernel, gamma = gamma, C = C)
    svm_virginica  = SVC(kernel =kernel, gamma = gamma, C = C)
    
    # train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)
    
    # 각 클래스 별 모델을 학습시키기 위하여 target을 one-hot encoding 해준다.
    y_train_encoded = pd.get_dummies(y_train) 
    
    # 각각의 model training
    svm_setosa.fit(X_train,y_train_encoded.iloc[:,0])
    svm_versicolor.fit(X_train,y_train_encoded.iloc[:,1])
    svm_virginica.fit(X_train,y_train_encoded.iloc[:,2])
    
    setosa_distance = svm_setosa.decision_function(X_test)
    versicolor_distance = svm_versicolor.decision_function(X_test)
    virginica_distance = svm_virginica.decision_function(X_test)
    
    # test 데이터 최종 예측
    pred = np.argmax(np.array([setosa_distance, versicolor_distance, virginica_distance]), axis=0)
    pred_eng = pd.Series(pred).replace({0:'setosa', 1:'versicolor', 2:'virginica'})
    
    return accuracy_score(y_test, pred_eng)


# 데이터 예측 방식에 대한 주석
# * ([svm_setosa가 예측한 결과, svm_versicolor가 예측한 결과, svm_virginica가 예측한 결과]) 순서로 표기
# 1. [0, 0, 0]으로 예측 -> decision_function 모두 음수 -> 그나마 큰 값으로 예측하자
# 2. [1, 0, 0], [0, 1, 0], [0, 0, 1]으로 예측 -> 하나만 decision_function 양수, 나머지 2개는 음수 -> 큰 값으로 예측
# 3. [1, 1, 0], [1, 0, 1], [0, 1, 1]으로 예측 -> 두 개는 decision_function 양수, 하나는 음수 -> 큰 값으로 예측 (거리 클수록 확실한 예측이라 판단)
# 4. [1, 1, 1]으로 예측 -> decision_function 모두 양수 -> 거리 가장 큰 값으로 예측
# -> 즉, 그냥 단순하게 decision_function의 값이 가장 크게 나온 것으로 예측하면 된다.

In [7]:
def hyperparameter_tunning(kernel, gamma_list, C_list, X, y):
    score_dict = {} 
    for gamma in tqdm(gamma_list):
        for C in C_list:
            score = one_rest_svm(kernel, gamma, C, X, y)
            param = '_'.join([kernel, str(gamma), str(C)])
            score_dict[param] = score
            
    return score_dict

In [8]:
# 1. scaling 하기 전의 데이터로 training
X = iris.iloc[:,:4]
y = iris.iloc[:,-1]

In [9]:
from tqdm import tqdm
kernel = 'rbf'
gamma_list = [0.5, 0.6, 0.7, 0.8, 0.9, 1, 2, 3, 4, 5, 6, 7, 8]
C_list = [0.01, 0.1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100, 150, 200]

# 여러가지 hyperparameter tuning 시도
# 여기서는 scaling하지 않은 데이터로 한 번 training 해보려고 한다.
# 그리고 test의 accuracy를 score_dict에 담아서 순위를 보려고 한다.

scores = hyperparameter_tunning(kernel, gamma_list, C_list, X, y)

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:02<00:00,  4.29it/s]


In [10]:
# rbf 커널에서는 gamma가 1, C가 0.1일 때 test 결과 좋았다.
# gamma가 클수록 overfitting, 작을수록 underfitting 되기 쉽다.
# C가 클수록 오분류된 데이터를 줄이기 위한 것에 집중하여 overfitting되기 쉽다.

pd.Series(scores).sort_values(ascending=False)[:10]

rbf_1_0.1      0.966667
rbf_0.8_0.1    0.966667
rbf_0.9_0.1    0.966667
rbf_0.5_1      0.933333
rbf_0.8_1      0.933333
rbf_0.9_1      0.933333
rbf_1_1        0.933333
rbf_0.6_0.1    0.933333
rbf_2_0.01     0.933333
rbf_2_0.1      0.933333
dtype: float64

In [11]:
gamma_list = list(np.arange(0.5, 1.5, 0.02))
C_list = list(np.arange(0.01, 0.5, 0.01))

scores = hyperparameter_tunning(kernel, gamma_list, C_list, X, y)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:19<00:00,  2.61it/s]


In [12]:
# 더 세밀하게 hyperparameter를 움직여 보았지만 score가 더 오르지 않았다
pd.Series(scores).sort_values(ascending=False)[:20]

rbf_0.6400000000000001_0.29000000000000004     0.966667
rbf_1.1600000000000006_0.01                    0.966667
rbf_0.9200000000000004_0.15000000000000002     0.966667
rbf_0.9200000000000004_0.14                    0.966667
rbf_0.6200000000000001_0.37                    0.966667
rbf_0.9200000000000004_0.13                    0.966667
rbf_0.9200000000000004_0.12                    0.966667
rbf_0.9200000000000004_0.09999999999999999     0.966667
rbf_1.2800000000000007_0.12                    0.966667
rbf_1.2800000000000007_0.11                    0.966667
rbf_1.2800000000000007_0.09999999999999999     0.966667
rbf_1.2800000000000007_0.09                    0.966667
rbf_1.2800000000000007_0.08                    0.966667
rbf_1.2800000000000007_0.06999999999999999     0.966667
rbf_1.2800000000000007_0.060000000000000005    0.966667
rbf_1.2800000000000007_0.05                    0.966667
rbf_1.2800000000000007_0.04                    0.966667
rbf_1.2800000000000007_0.03                    0

In [13]:
# 2. scaling한 데이터로 학습
standard_scaler = StandardScaler() 
X_scaled = standard_scaler.fit_transform(X)

In [14]:
kernel = 'rbf'
C_list = [0.01, 0.1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100, 150, 200]
gamma_list = [0.5, 0.6, 0.7, 0.8, 0.9, 1, 2, 3, 4, 5, 6, 7, 8]

scores = hyperparameter_tunning(kernel, gamma_list, C_list, X_scaled, y)

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:01<00:00,  6.86it/s]


In [15]:
# gamma가 0.8, C가 1일 때 accuracy가 높게 나와 이 근처에서 hyperparameter tuning을 더 해보려고 한다.
pd.Series(scores).sort_values(ascending=False)[:10]

rbf_0.8_1     0.966667
rbf_0.5_1     0.966667
rbf_0.7_1     0.966667
rbf_0.6_1     0.966667
rbf_0.8_2     0.933333
rbf_0.7_50    0.933333
rbf_0.6_70    0.933333
rbf_1_2       0.933333
rbf_0.9_2     0.933333
rbf_2_3       0.933333
dtype: float64

In [16]:
gamma_list = list(np.arange(0.5, 1.5, 0.02))
C_list = list(np.arange(0.5, 3, 0.05))

scores = hyperparameter_tunning(kernel, gamma_list, C_list, X_scaled, y)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:12<00:00,  3.97it/s]


In [17]:
# 더 세밀하게 hyperparameter를 움직여본 결과 test accuracy가 1이 나왔다 !!
pd.Series(scores).sort_values(ascending=False)[:20]

rbf_0.6400000000000001_0.9000000000000004    1.000000
rbf_0.5800000000000001_0.8000000000000003    1.000000
rbf_0.6000000000000001_0.8500000000000003    1.000000
rbf_0.6000000000000001_0.9000000000000004    1.000000
rbf_0.6000000000000001_0.9500000000000004    1.000000
rbf_0.6600000000000001_0.9000000000000004    1.000000
rbf_0.5800000000000001_0.8500000000000003    1.000000
rbf_0.6000000000000001_0.8000000000000003    1.000000
rbf_0.6200000000000001_0.9500000000000004    1.000000
rbf_0.6200000000000001_0.9000000000000004    1.000000
rbf_0.6200000000000001_0.8500000000000003    1.000000
rbf_0.7400000000000002_0.8000000000000003    0.966667
rbf_0.7400000000000002_0.7500000000000002    0.966667
rbf_0.7200000000000002_1.2500000000000007    0.966667
rbf_0.7400000000000002_0.7000000000000002    0.966667
rbf_0.7400000000000002_0.6500000000000001    0.966667
rbf_0.7400000000000002_0.6000000000000001    0.966667
rbf_0.7200000000000002_1.3000000000000007    0.966667
rbf_0.7400000000000002_0.950

In [18]:
svm_setosa     = SVC(kernel ='rbf', gamma = 0.6, C = 0.9)
svm_versicolor = SVC(kernel ='rbf', gamma = 0.6, C = 0.9)
svm_virginica  = SVC(kernel ='rbf', gamma = 0.6, C = 0.9)

# train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3)

# 각 클래스 별 모델을 학습시키기 위하여 target을 one-hot encoding 해준다.
y_train_encoded = pd.get_dummies(y_train) 

# 각각의 model training
svm_setosa.fit(X_train,y_train_encoded.iloc[:,0])
svm_versicolor.fit(X_train,y_train_encoded.iloc[:,1])
svm_virginica.fit(X_train,y_train_encoded.iloc[:,2])

setosa_distance = svm_setosa.decision_function(X_test)
versicolor_distance = svm_versicolor.decision_function(X_test)
virginica_distance = svm_virginica.decision_function(X_test)

# test 데이터 최종 예측
pred = np.argmax(np.array([setosa_distance, versicolor_distance, virginica_distance]), axis=0)
pred_eng = pd.Series(pred).replace({0:'setosa', 1:'versicolor', 2:'virginica'})

In [19]:
# standard scaling한 데이터 + rbf kernel + gamma 0.6 + C 0.9
# test size = 0.3 으로 random_state 없이 한 번 더 예측
# 역시 어느 정도 잘 나온다.

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred_eng)

array([[16,  0,  0],
       [ 0, 13,  1],
       [ 0,  0, 15]], dtype=int64)

In [20]:
accuracy_score(y_test, pred_eng)

0.9777777777777777